Bayes Rule Book:

https://www.bayesrulesbook.com/chapter-9.html

Materials from the Bayes Rule github:

https://github.com/bayes-rules/bayesrules

# Imports

In [1]:
pwd = %pwd
if pwd[:4] == '/mnt': #is azure instance
    %pip install pyreadr
    %pip install pyro-ppl
    %pip install statsmodels 
    %pip install graphviz

In [1]:
import math, pyreadr
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from scipy.stats import norm, beta, binom, mode
from os.path import exists

import pyro
import torch as t
import pyro.distributions as dist
import pyro.distributions.constraints as constraints
from pyro.infer import MCMC
from pyro.infer.mcmc.nuts import HMC, NUTS

device = t.device("cuda" if t.cuda.is_available() else "cpu")
t.set_default_tensor_type(t.FloatTensor)
if t.cuda.is_available():
    t.set_default_tensor_type(t.cuda.FloatTensor)


# Pyro Regression Example

Since the book's **Capital Bikeshare** example uses linear regression, we are going to quickly go through Pyro.ai 's tutorial on regression to get a hold of a working model. We can then adapt the model to our example.

Pyro's tutorial can be found here: https://pyro.ai/examples/bayesian_regression.html

Background:

>We would like to explore the relationship between topographic heterogeneity of a nation as measured by the Terrain Ruggedness Index (variable rugged in the dataset) and its GDP per capita. In particular, it was noted by the authors that terrain ruggedness or bad geography is related to poorer economic performance outside of Africa, but rugged terrains have had a reverse effect on income for African nations.

Features:

- `rugged`: quantifies the Terrain Ruggedness Index

- `cont_africa`: whether the given nation is in Africa

- `rgdppc_2000`: Real GDP per capita for the year 2000

## Data

In [107]:
data_url = "https://d2hg8soec8ck9v.cloudfront.net/datasets/rugged_data.csv"
df = pd.read_csv(data_url, encoding="ISO-8859-1")[["cont_africa", "rugged", "rgdppc_2000"]].dropna()
df.head()

,cont_africa,rugged,rgdppc_2000
2,1,0.858,1794.729
4,0,3.427,3703.113
7,0,0.769,20604.460
8,0,0.775,12173.680
9,0,2.688,2421.985


In [132]:
ff.create_distplot([df.rgdppc_2000, np.log(df.rgdppc_2000)*1000 - 5700], ['Actual', 'Log-Scaled'], bin_size=1000)

Since `rgdppc_2000` is highly skewed, we ought to log transform it.

In [133]:
df.rgdppc_2000 = np.log(df.rgdppc_2000)

In [143]:
fig = make_subplots(1,2, subplot_titles=("Non African Nations", "African Nations"))
fig.add_trace(go.Scatter(x=df.rugged.loc[df.cont_africa==0], y=df.rgdppc_2000.loc[df.cont_africa==0], mode='markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.rugged.loc[df.cont_africa==1], y=df.rgdppc_2000.loc[df.cont_africa==1], mode='markers'), row=1, col=2)

fig.update_layout(yaxis ={'range':[6, 11]}, yaxis2 ={'range':[6, 11]}, xaxis ={'range':[-.05, 7]}, xaxis2 ={'range':[-0.05, 7]})

fig

## Normal Regression

In [153]:
from torch import nn
from pyro.nn import PyroModule

In [225]:
df["cont_africa_x_rugged"] = df["cont_africa"] * df["rugged"]
data = t.tensor(df[['cont_africa', 'rugged', 'cont_africa_x_rugged', 'rgdppc_2000']].values, dtype=t.float)
x_data, y_data = data[:, :-1], data[:, -1]

In [226]:
# Regression model
linear_reg_model = PyroModule[nn.Linear](x_data.shape[1], 1)

# Define loss and optimize
loss_fn = t.nn.MSELoss(reduction='sum')
optim = t.optim.Adam(linear_reg_model.parameters(), lr=0.05)

In [227]:
def train():
    preds = linear_reg_model(x_data).squeeze(-1)
    loss = loss_fn(preds, y_data)

    optim.zero_grad()
    loss.backward()
    optim.step()
    return loss.item()

for i in range(1000):
    loss = train()
    if i % 100 == 0:
        print(f'[{i}] loss: {loss}')

print("Learned parameters:")
for name, param in linear_reg_model.named_parameters():
    print(name, param.data.numpy())

[0] loss: 16271.3984375
[100] loss: 1964.022216796875
[200] loss: 951.0186157226562
[300] loss: 584.5377197265625
[400] loss: 372.200439453125
[500] loss: 251.6545867919922
[600] loss: 190.98867797851562
[700] loss: 163.96116638183594
[800] loss: 153.26194763183594
[900] loss: 149.4932098388672
Learned parameters:
weight [[-1.8229191  -0.15761273  0.33216488]]
bias [9.1307]


In [228]:
fit = df.copy()
fit["mean"]         = linear_reg_model(x_data).detach().cpu().numpy()
african_nations     = fit[fit["cont_africa"] == 1]
non_african_nations = fit[fit["cont_africa"] == 0]

fig = make_subplots(1,2, subplot_titles=("Non African Nations", "African Nations"))

fig.add_trace(go.Scatter(x=non_african_nations["rugged"], y=non_african_nations["rgdppc_2000"], mode='markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=non_african_nations["rugged"], y=non_african_nations["mean"]), row=1, col=1)

fig.add_trace(go.Scatter(x=african_nations["rugged"], y=african_nations["rgdppc_2000"], mode='markers'), row=1, col=2)
fig.add_trace(go.Scatter(x=african_nations["rugged"], y=african_nations["mean"]), row=1, col=2)

fig

## Bayesian Regression with SVI

In [262]:
from pyro.nn import PyroSample
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO
from pyro.infer import Predictive

In [310]:
class BayesianRegression(PyroModule):

    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_dim, out_dim)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_dim, in_dim]).to_event(2))
        self.linear.bias   = PyroSample(dist.Normal(0., 10.).expand([out_dim]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mu = self.linear(x).squeeze(-1)

        with pyro.plate('data', x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma), obs=y)

        return mu

In [311]:
model = BayesianRegression(3, 1)
guide = AutoDiagonalNormal(model)

adam = pyro.optim.Adam({"lr": 0.03})
svi  = SVI(model, guide, adam, loss=Trace_ELBO())

In [312]:
pyro.clear_param_store()
for i in range(1000):
    loss = svi.step(x_data, y_data)
    if i % 100 == 0:
        print(f'[{i}] loss: {loss / len(data)}')

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

[0] loss: 4.717265826113084
[100] loss: 3.427936992224525
[200] loss: 3.047668890392079
[300] loss: 2.676327646479887
[400] loss: 1.9735149579889635
[500] loss: 1.4749756378286025
[600] loss: 1.482730678951039
[700] loss: 1.4689282305100384
[800] loss: 1.4625106853597305
[900] loss: 1.4708903635249417
AutoDiagonalNormal.loc Parameter containing:
tensor([-2.2420, -1.9279, -0.2308,  0.2759,  9.1272], requires_grad=True)
AutoDiagonalNormal.scale tensor([0.0590, 0.1199, 0.0374, 0.0766, 0.0706], grad_fn=<SoftplusBackward0>)


In [286]:
predictive = Predictive(model, guide=guide, num_samples=800, return_sites=("linear.weight", "obs", "_RETURN"))
samples = predictive(x_data)

def summary(samples):
    site_stats = {}
    for k, v in samples.items():
        site_stats[k] = {
            "mean": t.mean(v, 0),
            "std": t.std(v, 0),
            "5%": v.kthvalue(int(len(v) * 0.05), dim=0)[0],
            "95%": v.kthvalue(int(len(v) * 0.95), dim=0)[0],
        }
    return site_stats
pred_summary = summary(samples)

mu = pred_summary["_RETURN"]
y = pred_summary["obs"]
predictions = pd.DataFrame({
    "cont_africa": x_data[:, 0],
    "rugged": x_data[:, 1],
    "mu_mean": mu["mean"],
    "mu_perc_5": mu["5%"],
    "mu_perc_95": mu["95%"],
    "y_mean": y["mean"],
    "y_perc_5": y["5%"],
    "y_perc_95": y["95%"],
    "true_gdp": y_data,
})

In [287]:
african_nations     = predictions[predictions["cont_africa"] == 1].sort_values(by=["rugged"])
non_african_nations = predictions[predictions["cont_africa"] == 0].sort_values(by=["rugged"])

fig = make_subplots(1,2, subplot_titles=("Non African Nations", "African Nations"))

fig.add_trace(go.Scatter(x=non_african_nations["rugged"], y=non_african_nations["true_gdp"], mode='markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=non_african_nations["rugged"], y=non_african_nations["mu_perc_95"], fill='tonexty', line_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(x=non_african_nations["rugged"], y=non_african_nations["mu_perc_5"], fill='tonexty', line_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(x=non_african_nations["rugged"], y=non_african_nations["mu_mean"], line_color='orange'), row=1, col=1)

fig.add_trace(go.Scatter(x=african_nations["rugged"], y=african_nations["true_gdp"], mode='markers'), row=1, col=2)
fig.add_trace(go.Scatter(x=african_nations["rugged"], y=african_nations["mu_perc_95"], fill='tonexty', line_color='blue'), row=1, col=2)
fig.add_trace(go.Scatter(x=african_nations["rugged"], y=african_nations["mu_perc_5"], fill='tonexty', line_color='blue'), row=1, col=2)
fig.add_trace(go.Scatter(x=african_nations["rugged"], y=african_nations["mu_mean"], line_color='orange'), row=1, col=2)

fig

# Capital Bikeshare

## Data

In [2]:
data_url = 'https://raw.githubusercontent.com/ZachariahRosenberg/ames-iowa-housing-predictions/master/bikes.csv'
df = pd.read_csv(data_url)

In [3]:
df.head()

,Unnamed: 0,date,season,year,month,day_of_week,weekend,holiday,temp_actual,temp_feel,humidity,windspeed,weather_cat,rides
0,0,2011-01-01,winter,2011,Jan,Sat,True,no,57.399525,64.72625,80.5833,10.749882,categ2,654
1,1,2011-01-03,winter,2011,Jan,Mon,False,no,46.491663,49.04645,43.7273,16.636703,categ1,1229
2,2,2011-01-04,winter,2011,Jan,Tue,False,no,46.760000,51.09098,59.0435,10.739832,categ1,1454
3,3,2011-01-05,winter,2011,Jan,Wed,False,no,48.749427,52.63430,43.6957,12.522300,categ1,1518
4,4,2011-01-07,winter,2011,Jan,Fri,False,no,46.503324,50.79551,49.8696,11.304642,categ2,1362


In [4]:
px.scatter(x=df.temp_feel, y=df.rides, trendline="ols")

## Model

We want to predict the avg ridership per day, given *i* sampled days of ridership counts and high temperatures, {$(Y_{0}, x_{0}), (Y_{1}, x_{1}), ..., (Y_{n}, x_{n})$}. We don't know $\mu$, but we do have a prior belief of $\sigma$.

Effectively:

- $Y_{i}|\beta_{0}, \beta_{1}, \sigma \sim N(\mu_{i}, \sigma^2)$
- $~~~~~ \mu_{i} = \beta_{0} + \beta_{1}x_{i}$

In [5]:
temp_data = t.tensor(df.temp_feel.values, device=device, dtype=t.float).unsqueeze(1)
ride_data = t.tensor(df.rides.values, device=device, dtype=t.float)

### SVI

In [372]:
class BayesianRegression(PyroModule):

    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_dim, out_dim)
        self.linear.weight = PyroSample(dist.Normal(100., 40.).expand([out_dim, in_dim]).to_event(2))
        self.linear.bias   = PyroSample(dist.Normal(-3000., 1500.).expand([out_dim]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Exponential(0.08))
        mu = self.linear(x).squeeze(-1)

        with pyro.plate('data', x.shape[0]):
            pyro.sample("obs", dist.Normal(mu, sigma), obs=y)

        return mu

model = BayesianRegression(1, 1)
guide = AutoDiagonalNormal(model)

adam = pyro.optim.Adam({"lr": 0.001})
svi  = SVI(model, guide, adam, loss=Trace_ELBO())

In [373]:
pyro.clear_param_store()
losses = []
for i in range(20000):
    loss = svi.step(temp_data, ride_data)
    losses.append(loss)
    if i % 1000 == 0:
        print(f'[{i}] loss: {loss}')
        # early stopping if floor
        if i > 5000 and loss - np.mean(losses[-3000:]) < 1:
            print('early stopping')
            break

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

[0] loss: 4444.252912044525
[1000] loss: 4318.631947040558
[2000] loss: 4306.303215146065
[3000] loss: 4305.987208068371
[4000] loss: 4305.129846811295
[5000] loss: 4306.844422340393
[6000] loss: 4303.497738599777
early stopping
AutoDiagonalNormal.loc Parameter containing:
tensor([    7.1649,    98.3685, -3350.2612], requires_grad=True)
AutoDiagonalNormal.scale tensor([0.0364, 0.8181, 2.9658], grad_fn=<SoftplusBackward0>)


In [374]:
px.line(losses)

In [375]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df.temp_feel, y=df.rides, mode='markers')
)
for i in range(20):
    fig.add_trace(
        go.Scatter(x=df.temp_feel, y=model(temp_data).numpy())
    )
fig

### Vanilla Linear Regression

In [390]:
# Regression model
linear_reg_model = PyroModule[nn.Linear](temp_data.shape[1], 1)

# Define loss and optimize
loss_fn = t.nn.MSELoss()
optim = t.optim.Adam(linear_reg_model.parameters(), lr=0.05)

def train():
    preds = linear_reg_model(temp_data).squeeze(-1)
    loss = loss_fn(preds, ride_data)

    optim.zero_grad()
    loss.backward()
    optim.step()
    return loss.item()

losses = []
for i in range(10000):
    loss = train()
    losses.append(loss)
    if i % 250 == 0:
        print(f'[{i}] loss: {loss}')

print("Learned parameters:")
for name, param in linear_reg_model.named_parameters():
    print(name, param.data.numpy())

[0] loss: 14997641.0
[250] loss: 9590876.0
[500] loss: 6053767.0
[750] loss: 3905545.25
[1000] loss: 2717744.5
[1250] loss: 2133821.25
[1500] loss: 1885157.25
[1750] loss: 1795832.875
[2000] loss: 1769443.75
[2250] loss: 1763157.125
[2500] loss: 1761926.5
[2750] loss: 1761664.25
[3000] loss: 1761529.25
[3250] loss: 1761392.125
[3500] loss: 1761238.0
[3750] loss: 1761063.0
[4000] loss: 1760864.875
[4250] loss: 1760641.0
[4500] loss: 1760388.125
[4750] loss: 1760102.75
[5000] loss: 1759782.0
[5250] loss: 1759421.75
[5500] loss: 1759018.125
[5750] loss: 1758567.125
[6000] loss: 1758065.25
[6250] loss: 1757508.875
[6500] loss: 1756894.625
[6750] loss: 1756220.0
[7000] loss: 1755483.625
[7250] loss: 1754685.5
[7500] loss: 1753826.25
[7750] loss: 1752908.5
[8000] loss: 1751935.75
[8250] loss: 1750912.625
[8500] loss: 1749845.25
[8750] loss: 1748738.875
[9000] loss: 1747600.25
[9250] loss: 1746435.125
[9500] loss: 1745248.875
[9750] loss: 1744046.875
Learned parameters:
weight [[52.915794]]
b

*From book:* $−2194.24+82.16X$

In [391]:
px.line(losses)

In [392]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df.temp_feel, y=df.rides, mode='markers')
)

fig.add_trace(
    go.Scatter(x=df.temp_feel, y=linear_reg_model(temp_data).detach().numpy().squeeze(1))
)
fig

### MCMC

In [11]:
temp_data = t.tensor(df.temp_feel.values, device=device, dtype=t.float)
ride_data = t.tensor(df.rides.values, device=device, dtype=t.float)

#### Linear

In [53]:
def model(x, Y=None):
    beta_0 = pyro.sample('beta_0', dist.Normal(-2000, 1000))
    beta_1 = pyro.sample('beta_1', dist.Normal(100, 40))
    sigma  = pyro.sample('sigma', dist.Exponential(0.0008))

    mu = beta_0 + beta_1 * x

    with pyro.plate('data', len(x)):
        return pyro.sample('obs', dist.Normal(mu, sigma), obs=Y)

def conditioned_model(model, data, Y):
    return pyro.condition(model, data={'obs': Y})(data)

kernel = NUTS(conditioned_model)
mcmc = MCMC(kernel, num_samples=1000, warmup_steps=250)
mcmc.run(model, temp_data, ride_data)
mcmc.summary(prob=0.5)

Sample: 100%|██████████| 1250/1250 [01:33, 13.37it/s, step size=3.90e-02, acc. prob=0.946]


                mean       std    median     25.0%     75.0%     n_eff     r_hat
    beta_0  -2147.56    355.37  -2166.72  -2425.36  -1961.02    359.97      1.00
    beta_1     81.41      5.05     81.62     78.72     85.47    360.59      1.00
     sigma   1285.71     39.47   1285.18   1263.07   1317.16    266.27      1.00

Number of divergences: 0


Answer from book:

$y = -2194.24 + 82.16x$

where
- $\beta_{0} \sim (-2194, 362)$
- $\beta_{1} \sim (82, 5.15)$
- $\sigma \sim (3487, 80.4)$

I'd say our's was a success!

In [59]:
sample_betas = pd.DataFrame(mcmc.get_samples())[['beta_0', 'beta_1']]

def pull_sample():
    b0, b1 = sample_betas.sample(1).values[0]
    X = np.arange(45, 90)
    Y = [b0+b1*x for x in X]
    return X,Y

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df.temp_feel, y=df.rides, mode='markers')
)
for i in range(50):
    x, y = pull_sample()
    fig.add_trace(
        go.Scatter(x=x, y=y)
    )

fig.update_layout(showlegend=False)
fig

#### Quadratic

In [61]:
def quadratic_model(x, Y=None):
    beta_0 = pyro.sample('beta_0', dist.Normal(-2000, 1000))
    beta_1 = pyro.sample('beta_1', dist.Normal(100, 40))
    beta_2 = pyro.sample('beta_2', dist.Normal(0, 25))
    sigma  = pyro.sample('sigma', dist.Exponential(0.0008))

    mu = beta_0 + beta_1 * x + beta_2*x**2

    with pyro.plate('data', len(x)):
        return pyro.sample('obs', dist.Normal(mu, sigma), obs=Y)

def conditioned_model(model, data, Y):
    return pyro.condition(model, data={'obs': Y})(data)

kernel = NUTS(conditioned_model)
mcmc = MCMC(kernel, num_samples=1000, warmup_steps=250)
mcmc.run(quadratic_model, temp_data, ride_data)
mcmc.summary(prob=0.5)

Sample: 100%|██████████| 1250/1250 [04:31,  4.60it/s, step size=2.73e-02, acc. prob=0.934]


                mean       std    median     25.0%     75.0%     n_eff     r_hat
    beta_0  -2653.83    761.16  -2668.20  -3175.80  -2264.68    311.12      1.00
    beta_1     97.43     22.82     98.13     82.55    110.35    311.77      1.00
    beta_2     -0.12      0.17     -0.13     -0.23     -0.00    332.52      1.00
     sigma   1281.79     38.69   1281.26   1247.47   1298.61    401.81      1.00

Number of divergences: 0


In [62]:
sample_betas = pd.DataFrame(mcmc.get_samples())[['beta_0', 'beta_1', 'beta_2']]

def pull_sample():
    b0, b1, b2 = sample_betas.sample(1).values[0]
    X = np.arange(45, 90)
    Y = [b0+b1*x+b2*x**2 for x in X]
    return X,Y

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df.temp_feel, y=df.rides, mode='markers')
)
for i in range(10):
    x, y = pull_sample()
    fig.add_trace(
        go.Scatter(x=x, y=y)
    )

fig.update_layout(showlegend=False)
fig